In [ ]:
import requests
from bs4 import BeautifulSoup as BS

In [ ]:
url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222"
response = requests.get(url)
html = response.content
html = BS(html)

In [ ]:
div = html.find("div", attrs = {"class" : "board-type"})
li_list = div.find_all("li")

In [ ]:
print(li_list[0])

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>


In [ ]:
li_list[1]

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=036090">위지트</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('139094');">반도체 소모성 부품업체로 변신 중</a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
										
											-원
										
									
								</div>
<div class="line3 color-black">
									-
								</div>
<div class="line3">한국IR협의회</div>
<div class="line3">2022.12.01</div>
</li>

In [ ]:
li_list[-1]

<li><a href="#" onclick="linkPage(5); return false;">5</a></li>

In [ ]:
check_str = '<strong class="color-cate"><span class="bracket">'

In [ ]:
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))

True
False
False


In [ ]:
stock = li_list[1].find_all("a")[0].text
print(stock)

위지트


In [ ]:
title = li_list[1].find_all("a")[1].text
print(title)

반도체 소모성 부품업체로 변신 중


In [ ]:
price = li_list[1].find("div", attrs = {"class" : "line3"}).text
display(price)

'\n적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t-원\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t'

In [ ]:
price = price.replace("\r", "").replace("\t", "").replace("\n", "")
price = price.split(" ")[1].replace(",", "")[:-1]
display(price)

'-'

In [ ]:
opinion = li_list[1].find_all("div", attrs = {"class" : "line3"})[1].text
display(opinion)

'\r\n\t\t\t\t\t\t\t\t\t-\r\n\t\t\t\t\t\t\t\t'

In [ ]:
opinion = opinion.replace("\t", "").replace("\n", "").replace("\r", "")
display(opinion)

'-'

In [ ]:
trading_firm = li_list[1].find_all("div", attrs = {"class" : "line3"})[2].text
display(trading_firm)

'한국IR협의회'

In [ ]:
date = li_list[1].find_all("div", attrs = {"class" : "line3"})[3].text
display(date)

'2022.12.01'

In [ ]:
import pandas as pd
date = pd.to_datetime(date)
display(date)

Timestamp('2022-12-01 00:00:00')

# 단일 리포트 파싱

In [ ]:
def parsing_li(li):
  stock = li.find_all("a")[0].text  # 종목
  title = li.find_all("a")[1].text  # 리포트 제목
  price = li.find("div", attrs = {"class" : "line3"}).text # 적정 가격
  price = price.replace("\r", "").replace("\t", "").replace("\n", "")
  price = price.split(" ")[1].replace(",", "")[:-1]
  opinion = li.find_all("div", attrs = {"class" : "line3"})[1].text # 의견
  opinion = opinion.replace("\t", "").replace("\n", "").replace("\r", "")
  trading_firm = li.find_all("div", attrs = {"class" : "line3"})[2].text  #증권사
  date = li.find_all("div", attrs = {"class" : "line3"})[3].text  # 날짜
  date = pd.to_datetime(date)

  return [stock, title, price, opinion, trading_firm, date]

In [ ]:
print(parsing_li(li_list[1]))

['위지트', '반도체 소모성 부품업체로 변신 중', '-', '-', '한국IR협의회', Timestamp('2022-12-01 00:00:00')]


# 리포트 수집 및 출력

In [ ]:
data = list()
for li in li_list:
  if check_str in str(li):
    record = parsing_li(li)
    data.append(record)
  
data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])

In [ ]:
display(data.head(10))

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,위지트,반도체 소모성 부품업체로 변신 중,-,-,한국IR협의회,2022-12-01
1,하이브,조커 카드가 될 유료 구독 모델 도입,210000,매수,유안타증권,2022-12-01
2,로체시스템즈,견조한 성장세에도 매력적인 Valuation 주목,-,-,키움증권,2022-12-01
3,원텍,블록버스터 등극을 예고하는 올리지오,-,-,키움증권,2022-12-01
4,제이브이엠,해외 약국의 디지털화 수혜 지속 전망,-,-,키움증권,2022-12-01
5,두산퓨얼셀,"길었던 인고의 시간, 끝이 보인다",-,-,키움증권,2022-12-01
6,아이디스홀딩스,비대면 무인시대 4륜구동 르네상스,-,-,한국IR협의회,2022-11-30
7,아이패밀리에스씨,위기에도 성장을 기대할 수 있는 이유,-,-,한국IR협의회,2022-11-30
8,LG에너지솔루션,2023 년 질적 성장의 토대 마련,-,-,메리츠증권,2022-11-30
9,현대건설,업황 둔화 우려에도 기대 요인은 여전,55000,매수,유안타증권,2022-11-30


In [ ]:
import time
base_url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []

for page_no in range(1, 563):
  url = base_url.format(page_no)
  while(True):
    response = requests.get(url)
    if response.status_code == 200:
      time.sleep(1)
      break
    else:
      time.sleep(10 * 60)
  html = response.content
  html = BS(html)

  div = html.find("div", attrs = {"class":"board-type"})
  li_list = div.find_all("li")
  for li in li_list:
    if check_str in str(li):
      record = parsing_li(li)
      data.append(record)

data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])

In [ ]:
import time
today = time.strftime('%Y-%m-%d', time.localtime(time.time()))
data.to_csv("../{}종목리포트.txt".format(today), sep = "\t", encoding = "euc-kr", index = False)